In [29]:
import pandas as pd
import numpy as np

In [30]:
sf_data = pd.read_csv("/scratch/public/kushk/sf_with_homeloc_one_tweet_per_day.csv")


In [31]:
print("Original size: ", len(sf_data))

Original size:  14585347


In [32]:
#homedropped = len(sf_data[sf_data["home_tract"].isna()])
#print("Number of tracts without home_tract: " ,  homedropped)

In [33]:
#tractdropped = (sf_data[sf_data["tract"].isna()]) 
#print("Number of tracts without tract: " , tractdropped )

In [34]:
sf_data = sf_data[~sf_data["home_tract"].isna()]
sf_data = sf_data[~sf_data["tract"].isna()]
sf_data = sf_data[["date", "tract" , "home_tract"]]

In [35]:
print("Final size after dropping nulls: ", len(sf_data))

Final size after dropping nulls:  14585347


In [36]:
nn_data = pd.read_csv("/scratch/public/kushk/sfbay_13county_nearest_neigbbor_new.csv")

In [37]:
nn_data.head()

,SRC_GEOID,NBR_GEOID,LENGTH,NODE_COUNT
0,6001400100,6001404300,11.573519,0
1,6001400100,6001404400,4424.828741,0
2,6001400100,6001421600,477.810654,0
3,6001400100,6001422600,1282.939488,0
4,6001400100,6001422700,553.272946,0


In [38]:
sf_data.head()

,date,tract,home_tract
0,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09
1,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09
2,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09
3,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09
4,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09


In [39]:
neighbors = {}
for src,dest in zip(nn_data["SRC_GEOID"] , nn_data["NBR_GEOID"]):
    if src not in neighbors:
        neighbors[src] = set()
    neighbors[src].add(dest)

In [40]:
#If tweet is not in either tract or not home 
sf_data= sf_data[sf_data["tract"].isin(neighbors) & sf_data["home_tract"].isin(neighbors) ]

In [41]:
def processNeighbors(row):
    tract = int(row["tract"])
    home_tract = int(row["home_tract"])
    if(tract == home_tract):
        return "home"
    elif(home_tract in neighbors and tract in neighbors[home_tract]):
        return "neighbor"
    else:
        return "non-neighbor"

In [42]:
%%time
sf_data["locality"] = sf_data.apply(processNeighbors, axis=1)
sf_data["locality"].value_counts()

CPU times: user 11min 17s, sys: 3.06 s, total: 11min 20s
Wall time: 11min 21s


In [43]:
sf_data = sf_data[sf_data["locality"] !=  "not in set?"]

In [44]:
sf_data["locality"].value_counts()

non-neighbor    7005550
home            5126676
neighbor        1723411
Name: locality, dtype: int64

In [45]:
sf_data.head()

,date,tract,home_tract,locality
0,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09,non-neighbor
1,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09,home
2,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09,home
3,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09,non-neighbor
4,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09,home


In [46]:
overall = sf_data.groupby(["tract" , "locality" ]).size().reset_index().rename(columns={0:'count'})

In [47]:
sf_data['hour'] = sf_data['date'].str[11:13]
dayvalues = ['04', '05', '06', '07', '08', '09', '10', '11', '12', '13','14', '15', '16', '17', '18']
sf_data['day'] = sf_data.hour.map(lambda x: 1 if x in dayvalues else 0)

In [48]:
dayNight = sf_data.groupby(["tract" , "locality", "day"]).size().reset_index().rename(columns={0:'count'})
dayNight.head()

,tract,locality,day,count
0,6.001400e+09,home,0,285
1,6.001400e+09,home,1,323
2,6.001400e+09,neighbor,0,195
3,6.001400e+09,neighbor,1,84
4,6.001400e+09,non-neighbor,0,1577


In [49]:
sf_data['hour'] 

0           00
1           00
2           00
3           00
4           00
5           00
6           00
7           00
8           00
9           00
10          00
11          00
12          00
13          00
14          00
15          00
16          00
17          00
18          00
19          07
20          16
21          18
22          21
23          03
24          10
25          00
26          00
27          00
28          00
29          00
            ..
14585316    16
14585317    16
14585318    16
14585319    16
14585320    16
14585321    17
14585322    17
14585323    17
14585324    17
14585325    17
14585326    16
14585327    16
14585328    16
14585329    16
14585330    17
14585331    17
14585332    17
14585333    17
14585334    17
14585335    17
14585336    17
14585337    16
14585338    16
14585339    17
14585340    17
14585341    17
14585342    17
14585344    17
14585345    17
14585346    17
Name: hour, Length: 13855637, dtype: object

In [50]:
sf_data['date_helper'] = sf_data['date'].str[0:10]

valuesweekday= [1,2,3]
valuesweekend = [6, 5]
before_18 = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', 
             '10', '11', '12', '13', '14', '15', '16', '17', '18']

def weekday(row):
    day = int(pd.to_datetime(row["date_helper"]).weekday())
    if day in valuesweekday:
        return 1
    elif day == 0 & (row['hour'] not in ['00', '01', '02', '03']):
        return 1
    elif day == 4 & (row['hour'] in before_18):
        return 1
    else:
        return 0
sf_data['weekday'] = sf_data.apply(weekday , axis=1)



#sf_data['weekday'] = sf_data.date_helper.map(lambda x: 0 if pd.to_datetime(x).weekday() > 4 else 1)


In [51]:
weekDay = sf_data.groupby(["tract" , "locality", "weekday"]).size().reset_index().rename(columns={0:'count'})
weekDay.head()

,tract,locality,weekday,count
0,6.001400e+09,home,0,256
1,6.001400e+09,home,1,352
2,6.001400e+09,neighbor,0,126
3,6.001400e+09,neighbor,1,153
4,6.001400e+09,non-neighbor,0,1492


In [52]:
tracts = list(set(sf_data["tract"]))

#sf_data./

In [53]:
%%time
counts = [[] , [] , []]
dayCounts = [[[], []] , [[],[]] , [[],[]]]
weekDayCounts = [[[], []] , [[],[]] , [[],[]]]
locs = ["home" , "neighbor" , "non-neighbor"]


for tract in tracts:
    one_tract = overall[overall["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        if(len(one_loc) > 0):
            counts[i].append(one_loc["count"].iloc[0])
        else:
            counts[i].append(0)
        i+=1
        
for tract in tracts:
    one_tract = dayNight[dayNight["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["day"] == val]
            if(len(one_val) > 0):
                dayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                dayCounts[i][val].append(0)
        i+=1
                         
for tract in tracts:
    one_tract = weekDay[weekDay["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["weekday"] == val]
            if(len(one_val) > 0):
                weekDayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                weekDayCounts[i][val].append(0)
        i+=1

CPU times: user 51.6 s, sys: 96 ms, total: 51.7 s
Wall time: 51.7 s


In [54]:
sf_final = pd.DataFrame({"tract": pd.Series(tracts)})
i=0
for loc in locs:
    sf_final[loc] = pd.Series(counts[i])
    sf_final[loc + " day"] = pd.Series(dayCounts[i][1])
    sf_final[loc + " night"] = pd.Series(dayCounts[i][0])
    sf_final[loc + " weekday"] = pd.Series(weekDayCounts[i][1])
    sf_final[loc + " weekend"] = pd.Series(weekDayCounts[i][0])
    i+=1

In [55]:
sf_final

,tract,home,home day,home night,home weekday,home weekend,neighbor,neighbor day,neighbor night,neighbor weekday,neighbor weekend,non-neighbor,non-neighbor day,non-neighbor night,non-neighbor weekday,non-neighbor weekend
0,6.067003e+09,1097,649,448,650,447,95,53,42,54,41,1196,550,646,653,543
1,6.013354e+09,528,249,279,311,217,123,75,48,68,55,3492,1517,1975,1828,1664
2,6.013354e+09,1625,891,734,952,673,334,136,198,171,163,3273,1514,1759,1777,1496
3,6.075015e+09,2332,1257,1075,1297,1035,835,350,485,389,446,6946,2524,4422,3253,3693
4,6.075023e+09,1130,685,445,660,470,267,143,124,137,130,1502,672,830,804,698
5,6.001410e+09,1480,821,659,858,622,317,163,154,175,142,1550,685,865,849,701
6,6.081602e+09,1206,679,527,705,501,1339,601,738,736,603,7408,2460,4948,3787,3621
7,6.081602e+09,2707,1594,1113,1582,1125,774,368,406,425,349,3234,1432,1802,1682,1552
8,6.081602e+09,1808,1031,777,1030,778,1989,696,1293,1104,885,14494,4301,10193,7333,7161
9,6.081602e+09,2393,1268,1125,1334,1059,798,400,398,462,336,1955,928,1027,950,1005


In [56]:
sf_final.to_csv("Absolute_new_1031_13county_fulldrop_oneuser.csv")

In [ ]:
sum(sf_final["home"]) + sum(sf_final["non-neighbor"])